# Spark practical work
### Melen Laclais, Carlos Manzano Izquierdo

## Load of training data 

In [30]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, isnan

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Flight_Delay_EDA_Analysis") \
    .config("spark.driver.memory", "8g") \
    .master("local[*]") \
    .getOrCreate()

# Load Training Data (2006 + 2007)
print("Loading raw flight data (2006-2007)... This may take a moment due to schema inference...")

# We use inferSchema=True to see the actual data types Spark detects
# We use nullValue="NA" because the documentation states "NA" is used for nulls
flights_raw = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("nullValue", "NA") \
    .csv(["../training_data/flight_data/2006.csv.bz2", "../training_data/flight_data/2007.csv.bz2"])

# Load auxiliary tables for inspection
planes_df = spark.read.option("header", "true").option("inferSchema", "true").option("nullValue", "NA").csv("../training_data/flight_data/plane-data.csv")
airports_df = spark.read.option("header", "true").option("inferSchema", "true").option("nullValue", "NA").csv("../training_data/flight_data/airports.csv")

print(f"Total flights loaded: {flights_raw.count():,}")

Loading raw flight data (2006-2007)... This may take a moment due to schema inference...


Total flights loaded: 14,595,137


## Dataset exploration, analysis and processing

First, lets start by analyzing the schema of the dataset

In [31]:

print("--- Inferred Schema ---")
flights_raw.printSchema()

#(using Pandas for better formatting in Jupyter)
print("--- Data Sample (First 5 rows) ---")
flights_raw.limit(5).toPandas()

--- Inferred Schema ---
root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: integer (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: i

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2006,1,11,3,743,745,1024,1018,US,343,...,45,13,0,None,0,0,0,0,0,0
1,2006,1,11,3,1053,1053,1313,1318,US,613,...,27,19,0,None,0,0,0,0,0,0
2,2006,1,11,3,1915,1915,2110,2133,US,617,...,4,11,0,None,0,0,0,0,0,0
3,2006,1,11,3,1753,1755,1925,1933,US,300,...,16,10,0,None,0,0,0,0,0,0
4,2006,1,11,3,824,832,1015,1015,US,765,...,27,12,0,None,0,0,0,0,0,0



### Data Cleaning:  Delete Forbidden variables


Now the first step that we will take is to delete all of the variables whose information would not be available during inference, the "Forbidden" variables:

* ArrTime
* ActualElapsedTime
* AirTime
* TaxiIn
* Diverted
* CarrierDelay
* WeatherDelay
* NASDelay
* SecurityDelay
* LateAircraftDelay

In [32]:

# These variables contain future information known only after landing.
forbidden_vars = [
    "ArrTime",
    "ActualElapsedTime",
    "AirTime",
    "TaxiIn",
    "Diverted",
    "CarrierDelay",
    "WeatherDelay",
    "NASDelay",
    "SecurityDelay",
    "LateAircraftDelay"
]

# Drop the forbidden columns
flights_clean = flights_raw.drop(*forbidden_vars)

print("--- Structure after removing forbidden variables ---")
flights_clean.printSchema()

print("Ensure these variables are not anymore present:\n")

error_found = False
for var in forbidden_vars:
    if var in flights_clean.columns:
        print(f"ERROR: {var} is still present!")
        error_found = True
        
if error_found:
    print("Some forbidden variables are still present. Please check the code.")
else:
    print("All forbidden variables successfully removed.")


--- Structure after removing forbidden variables ---
root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)

Ensure these variables are not anymore present:

All forbidden variables successfully removed.



### Data Cleaning: Delete canceled flights and associated columns

The first cleaning step we must do is removing all the rows whose flights were cancelled, and delete the columns which express if the flight were canceled, as they won't have useful information anymore.

In [33]:
flights_clean = flights_clean.filter("Cancelled == 0") \
                            .drop("Cancelled", "CancellationCode")
                            
flights_clean.printSchema()                            
print(f"Total valid flights for training: {flights_clean.count():,}")


root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)



Total valid flights for training: 14,312,455


### Data Cleaning: Null handling

Now we will keep assesing the quality of the features that we have available, to verify which techniques of imputation and deletion we should follow. We will also evaluate if we need to transform into categorical some of the variables,  and the consistence showed in their values. 


In [34]:
print("--- Missing Values Analysis ---")
# Calculate count of nulls for each column
null_counts = flights_clean.select([count(when(col(c).isNull(), c)).alias(c) for c in flights_clean.columns])
null_counts.show(truncate=False)

--- Missing Values Analysis ---


+----+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|CRSElapsedTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiOut|
+----+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+
|0   |0    |0         |0        |0      |0         |0         |0            |0        |0      |727           |33365   |0       |0     |0   |0       |0      |
+----+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+



The only columns that contain null values are **CRSElapsedTime** and **ArrDelay**, with **727** and **33**,**365** missing values respectively.

When dealing with the issue where a lot ot observations have a null value in ArrDelay, it maybe could be consequence of not having filtered the rows whose flight was diverted.It could make sense that  this ones does not have any value setted in the colum ArrDelay, since the flight did not arrive to his original destiny. Thus, we must check if this hypothesis is true.


In [35]:
print("---  Hypothesis Verification (ArrDelay Nulls vs Diverted) ---")

# We check the raw data to see the relationship between Null ArrDelay and Diverted status
flights_raw.filter("ArrDelay IS NULL AND Cancelled == 0") \
           .select("ArrDelay", "Diverted", "Cancelled") \
           .groupBy("Diverted") \
           .count() \
           .show()

---  Hypothesis Verification (ArrDelay Nulls vs Diverted) ---


+--------+-----+
|Diverted|count|
+--------+-----+
|       1|33365|
+--------+-----+



We can confirm our hypothesis since the number of rows which satisfy the condition is equal to the number of nulls we observed earlier in the target variable. Then, we have to process again the flights raw data in order to obtain the clean dataset without nulls in ArrDelay as consequence of flights diverted.

In [36]:
# We must filter out 'Diverted' flights BEFORE dropping the 'Diverted' column.
print("--- Applying Fix (Filtering Diverted flights ---")

# Re-creating flights_clean with the correct logic:
# Filter Cancelled AND Diverted -> Then drop forbidden variables
flights_clean = flights_raw.filter("Cancelled == 0 AND Diverted == 0") \
                           .drop(*forbidden_vars) \
                           .drop("Cancelled", "CancellationCode")
                           
print("--- Schema check after applying fix: ---")
flights_clean.printSchema()

# Checking if 'ArrDelay' still has nulls after the fix.
print("--- Final Verification (Nulls in ArrDelay) ---")

flights_clean.select(
    count(when(col("ArrDelay").isNull(), "ArrDelay")).alias("ArrDelay_Null_Count")
).show()

# Verify if the number of columns in flights_clean matches the expected count
# (Raw columns - Forbidden variables - 2 cancellation columns)
expected_count = len(flights_raw.columns) - len(forbidden_vars) - 2
actual_count = len(flights_clean.columns)

print(f"Number of columns in flights_clean: {actual_count}")
print(f"Expected number of columns: {expected_count}")

if actual_count == expected_count:
    print("Verification successful: The column count matches the expected reduction.")
else:
    print("Verification failed: The column count does not match.")



--- Applying Fix (Filtering Diverted flights ---
--- Schema check after applying fix: ---
root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)

--- Final Verification (Nulls in ArrDelay) ---


+-------------------+
|ArrDelay_Null_Count|
+-------------------+
|                  0|
+-------------------+

Number of columns in flights_clean: 17
Expected number of columns: 17
Verification successful: The column count matches the expected reduction.


Now that we have fixed the problem of the nulls in our target variable, we have to manage the **nulls of CRSElapsedTime**. So lets impute the nulls, but , how ?. We thinked about calculating the difference between the columns **CrSArrTime and CRSDepTime**, but since we are working with data from flights, this would be difficult since there are differente time zones that would complicate to calcualate this value in a straightforward way. 

Thus, we have decided to impute this values with the average of the values present in other observation for the flights with the same origin and destination.

In [37]:
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, col, coalesce, round

print("--- Imputing CRSElapsedTime using Route Average ---")

# Define the Window: "Look at all flights sharing the same Origin and Dest"
route_window = Window.partitionBy("Origin", "Dest")

# Calculate the average scheduled duration for that specific route
# We use 'round' to keep it as integer minutes
avg_route_duration = round(avg("CRSElapsedTime").over(route_window))

# Apply Imputation
# If CRSElapsedTime is null, use the Route Average.
# If it's still null (e.g., a unique route with only 1 flight that is null), we drop it later.
flights_clean = flights_clean.withColumn(
    "CRSElapsedTime",
    coalesce(col("CRSElapsedTime"), avg_route_duration)
)

# Final Cleanup
# Drop any remaining nulls (routes that had NO valid data at all to learn from)
flights_clean = flights_clean.dropna(subset=["CRSElapsedTime"])

# Verification
print("--- Final Verification ---")
flights_clean.select(
    count(when(col("CRSElapsedTime").isNull(), 1)).alias("Nulls_After_Smart_Imputation")
).show()

# Show a sample of how it worked (e.g. for a specific route)
flights_clean.filter("Origin == 'JFK' AND Dest == 'LAX'") \
             .select("Origin", "Dest", "CRSDepTime", "CRSArrTime", "CRSElapsedTime") \
             .limit(5).show()

--- Imputing CRSElapsedTime using Route Average ---
--- Final Verification ---


+----------------------------+
|Nulls_After_Smart_Imputation|
+----------------------------+
|                           0|
+----------------------------+



+------+----+----------+----------+--------------+
|Origin|Dest|CRSDepTime|CRSArrTime|CRSElapsedTime|
+------+----+----------+----------+--------------+
|   JFK| LAX|      1400|      1709|         369.0|
|   JFK| LAX|      1400|      1709|         369.0|
|   JFK| LAX|      1400|      1709|         369.0|
|   JFK| LAX|      1400|      1709|         369.0|
|   JFK| LAX|      1400|      1709|         369.0|
+------+----+----------+----------+--------------+



So lets evaluate finally the null count on this dataset before making further analisys and transforamtions

In [38]:
# Calculate count of nulls for each column
null_counts = flights_clean.select([count(when(col(c).isNull(), c)).alias(c) for c in flights_clean.columns])
null_counts.show(truncate=False)

+----+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|CRSElapsedTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiOut|
+----+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+
|0   |0    |0         |0        |0      |0         |0         |0            |0        |0      |0             |0       |0       |0     |0   |0       |0      |
+----+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+



In [ ]:
# Save the cleaned dataframe to a parquet file for future use
# flights_clean.write.mode("overwrite").parquet("../check_point")

# print("Dataframe successfully saved to ../check_point")

Dataframe successfully saved to ../check_point


### Data Cleaning: Removal of High-Cardinality Identifiers
We proceed to remove specific identifiers that hinder model generalization and introduce noise. FlightNum is discarded as it is a non-ordinal administrative label whose predictive value is effectively redundant, being already better captured by the combination of route, carrier, and schedule information. Similarly we will remove the raw TailNum to prevent the model from overfitting to specific aircraft histories and to avoid the computational overhead associated with processing thousands of unique categorical levels(But this will be done in a few cells, since we need TailNum to join with the plane-data csv file)

In [39]:

flights_clean = flights_clean.drop("FlightNum")


### Feature Engineering: Join dataset with other files to enrich features
(*exploring additional datasets to try to find additional relevant
information*)


We have available in the dataset the csv files airports, carriers and  plane-data which we can use to enrich our dataset with more relevant features. We will explore them to asses which new variables we could obtain from joining them to our data.

In [40]:
# Explore additional datasets 
additional_datasets = {
    "planes": "../training_data/flight_data/plane-data.csv",
    "airports": "../training_data/flight_data/airports.csv",
    "carriers": "../training_data/flight_data/carriers.csv"
}
for name, path in additional_datasets.items():
    df = spark.read.option("header", "true").option("inferSchema", "true").option("nullValue", "NA").csv(path)
    print(f"--- {name.capitalize()} Dataset Schema ---")
    df.printSchema()
    print(f"--- {name.capitalize()} Dataset Sample (First 5 rows) ---")
    df.limit(5).show()

--- Planes Dataset Schema ---
root
 |-- tailnum: string (nullable = true)
 |-- type: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- issue_date: string (nullable = true)
 |-- model: string (nullable = true)
 |-- status: string (nullable = true)
 |-- aircraft_type: string (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- year: string (nullable = true)

--- Planes Dataset Sample (First 5 rows) ---
+-------+----+------------+----------+-----+------+-------------+-----------+----+
|tailnum|type|manufacturer|issue_date|model|status|aircraft_type|engine_type|year|
+-------+----+------------+----------+-----+------+-------------+-----------+----+
| N050AA|NULL|        NULL|      NULL| NULL|  NULL|         NULL|       NULL|NULL|
| N051AA|NULL|        NULL|      NULL| NULL|  NULL|         NULL|       NULL|NULL|
| N052AA|NULL|        NULL|      NULL| NULL|  NULL|         NULL|       NULL|NULL|
| N054AA|NULL|        NULL|      NULL| NULL|  NULL|         N

We will only use plane-data and airports csv's, since the information available in carriers does not give us useful information for our prediction. We can appreciate that apparantly, plane-data csv have some empty observations. Lets verify that.

In [41]:
#Verify null observations and columns in additional datasets
for name, path in additional_datasets.items():
    df = spark.read.option("header", "true").option("inferSchema", "true").option("nullValue", "NA").csv(path)
    print(f"--- {name.capitalize()} Dataset Null Values Analysis ---")
    null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
    null_counts.show(truncate=False)

--- Planes Dataset Null Values Analysis ---
+-------+----+------------+----------+-----+------+-------------+-----------+----+
|tailnum|type|manufacturer|issue_date|model|status|aircraft_type|engine_type|year|
+-------+----+------------+----------+-----+------+-------------+-----------+----+
|0      |549 |549         |549       |549  |549   |549          |549        |549 |
+-------+----+------------+----------+-----+------+-------------+-----------+----+

--- Airports Dataset Null Values Analysis ---
+----+-------+----+-----+-------+---+----+
|iata|airport|city|state|country|lat|long|
+----+-------+----+-----+-------+---+----+
|0   |0      |12  |12   |0      |0  |0   |
+----+-------+----+-----+-------+---+----+

--- Carriers Dataset Null Values Analysis ---
+----+-----------+
|Code|Description|
+----+-----------+
|1   |0          |
+----+-----------+



Now that we have seen that there are some nulls present in the auxiliar tables, we will have to manage that before joining the tables. 

Thus, We will  first perform a quality check on the auxiliary tables, removing records with missing critical information—such as aircraft manufacturing year or airport geographical coordinates—to prevent propagating noise.

We will then execute a series of left joins to incorporate the aircraft's manufacturing year and the geographical context (state and coordinates) for both the origin and destination airports. Finally, we implement a post-join validation step by removing flight records that failed to match with these auxiliary tables. This ensures data integrity and a consistent feature set for the subsequent creation of engineered variables.

In [45]:
from pyspark.sql.functions import col, broadcast

# load dataframes 
planes_df = spark.read.option("header", "true").option("inferSchema", "true").option("nullValue", "NA").csv("../training_data/flight_data/plane-data.csv")
airports_df = spark.read.option("header", "true").option("inferSchema", "true").option("nullValue", "NA").csv("../training_data/flight_data/airports.csv")



# 1. Pre-cleaning 
# ----------------------------------------------------------
planes_clean = planes_df.dropna(subset=["year", "manufacturer"]) \
                        .select(col("tailnum").alias("TailNum_Ref"), 
                                col("year").alias("PlaneYear"),
                                col("manufacturer").alias("PlaneManufacturer"))

airports_clean = airports_df.dropna(subset=["city", "state"]) \
                            .select(col("iata").alias("iata_ref"), 
                                    col("city"), col("state"), 
                                    col("lat"), col("long"))

# 2. Executing Joins
# ----------------------------------------------------------


# Plane data
enriched_df = flights_clean.join(broadcast(planes_clean), 
                                 flights_clean.TailNum == planes_clean.TailNum_Ref, "left")

# Origin Airport data
enriched_df = enriched_df.join(
    broadcast(airports_clean.select(col("iata_ref").alias("O_ref"), 
                                    col("city").alias("Origin_City"),
                                    col("state").alias("Origin_State"), 
                                    col("lat").alias("Origin_Lat"), 
                                    col("long").alias("Origin_Long"))),
    col("Origin") == col("O_ref"), "left"
)

# Destination Airport data
enriched_df = enriched_df.join(
    broadcast(airports_clean.select(col("iata_ref").alias("D_ref"), 
                                    col("city").alias("Dest_City"),
                                    col("state").alias("Dest_State"), 
                                    col("lat").alias("Dest_Lat"), 
                                    col("long").alias("Dest_Long"))),
    col("Dest") == col("D_ref"), "left"
)

# 3. Data Loss Verification (The Diagnostic)
# ----------------------------------------------------------
print("--- Data Coverage Analysis (Missing Matches) ---")

total_flights = enriched_df.count()

# Count nulls for each source
stats = enriched_df.select(
    count(when(col("PlaneYear").isNull(), 1)).alias("Missing_Plane_Info"),
    count(when(col("Origin_City").isNull(), 1)).alias("Missing_Origin_Info"),
    count(when(col("Dest_City").isNull(), 1)).alias("Missing_Dest_Info")
).collect()[0]

print(f"Total Rows: {total_flights:,}")
print(f"Flights without Plane metadata: {stats['Missing_Plane_Info']:,} ({stats['Missing_Plane_Info']/total_flights:.2%})")
print(f"Flights without Origin metadata: {stats['Missing_Origin_Info']:,} ({stats['Missing_Origin_Info']/total_flights:.2%})")
print(f"Flights without Destination metadata: {stats['Missing_Dest_Info']:,} ({stats['Missing_Dest_Info']/total_flights:.2%})")

# 4. Final Cleanup (Turning Left Join into "Inner Join" logic)
# ----------------------------------------------------------
enriched_df = enriched_df.dropna(subset=["PlaneYear", "Origin_City", "Dest_City"])
enriched_df = enriched_df.drop("TailNum_Ref", "O_ref", "D_ref")

print(f"\nFinal training set size: {enriched_df.count():,}")

print(f"Final column count: {len(enriched_df.columns)}")
enriched_df.printSchema()

print("--- Enriched Data Sample (First 5 rows) ---")
enriched_df.limit(5).show()

# 5. Null value analysis after enrichment
print("--- Null Values Analysis after Enrichment ---")
null_counts = enriched_df.select([count(when(col(c).isNull(), c)).alias(c) for c in enriched_df.columns])
null_counts.show(truncate=False)

--- Data Coverage Analysis (Missing Matches) ---


Total Rows: 14,279,090
Flights without Plane metadata: 2,350,705 (16.46%)
Flights without Origin metadata: 9,926 (0.07%)
Flights without Destination metadata: 9,968 (0.07%)



Final training set size: 11,910,044
Final column count: 26
root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- PlaneYear: string (nullable = true)
 |-- PlaneManufacturer: string (nullable = true)
 |-- Origin_City: string (nullable = true)
 |-- Origin_State: string (nullable = true)
 |-- Origin_Lat: double (nullable = true)
 |-- Origin_Long: double (nullable = true)
 |-- Dest_City: stri

+----+-----+----------+---------+-------+----------+----------+-------------+-------+--------------+--------+--------+------+----+--------+-------+---------+-----------------+-----------+------------+-----------+------------+---------+----------+-----------+------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|UniqueCarrier|TailNum|CRSElapsedTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiOut|PlaneYear|PlaneManufacturer|Origin_City|Origin_State| Origin_Lat| Origin_Long|Dest_City|Dest_State|   Dest_Lat|   Dest_Long|
+----+-----+----------+---------+-------+----------+----------+-------------+-------+--------------+--------+--------+------+----+--------+-------+---------+-----------------+-----------+------------+-----------+------------+---------+----------+-----------+------------+
|2006|    1|         2|        1|   1032|      1040|      1200|           XE| N27506|          80.0|       6|      -8|   ABE| CLE|     339|     27|     1999|          EMBRAER|  Allento

+----+-----+----------+---------+-------+----------+----------+-------------+-------+--------------+--------+--------+------+----+--------+-------+---------+-----------------+-----------+------------+----------+-----------+---------+----------+--------+---------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|UniqueCarrier|TailNum|CRSElapsedTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiOut|PlaneYear|PlaneManufacturer|Origin_City|Origin_State|Origin_Lat|Origin_Long|Dest_City|Dest_State|Dest_Lat|Dest_Long|
+----+-----+----------+---------+-------+----------+----------+-------------+-------+--------------+--------+--------+------+----+--------+-------+---------+-----------------+-----------+------------+----------+-----------+---------+----------+--------+---------+
|0   |0    |0         |0        |0      |0         |0         |0            |0      |0             |0       |0       |0     |0   |0       |0      |0        |0                |0          |0           |0       

### Feature Engineering: Agreggation and mixing of columns in order to create new columns that could be useful 

With the external datasets successfully integrated, we now transform raw data into more meaningful predictors for the model. We calculate the aircraft's age—derived from the flight year and manufacture year—to capture potential mechanical reliability trends. To account for daily operational cycles and peak congestion, we extract the scheduled departure hour from the timestamp. Additionally, we create a binary indicator to distinguish weekend traffic dynamics from business-day patterns. To conclude this stage, we drop the raw manufacture year and aircraft identifiers, refining our feature space to focus on generalized signals rather than specific registrations.

In [46]:
from pyspark.sql.functions import col, floor, expr

# 1. PlaneAge: Usamos try_cast para que los "None" pasen a ser NULL silenciosamente
enriched_df = enriched_df.withColumn(
    "PlaneAge", 
    (col("Year") - expr("try_cast(PlaneYear AS INT)")).cast("int")
)

# 2. DepHour: Aplicamos try_cast también por seguridad si CRSDepTime viene sucio
enriched_df = enriched_df.withColumn(
    "DepHour", 
    floor(expr("try_cast(CRSDepTime AS INT)") / 100).cast("int")
)

# 3. IsWeekend: Binary flag (DayOfWeek suele ser INT, pero por si acaso...)
enriched_df = enriched_df.withColumn(
    "IsWeekend",
    (expr("try_cast(DayOfWeek AS INT)") > 5).cast("int")
)

# 4. Final Cleanup
enriched_df = enriched_df.drop("TailNum", "PlaneYear")

print("--- New Features Created with Spark 4.0 Safety ---")
enriched_df.printSchema()
enriched_df.select("Year", "PlaneAge", "DepHour", "IsWeekend").show(5)

--- New Features Created with Spark 4.0 Safety ---
root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- PlaneManufacturer: string (nullable = true)
 |-- Origin_City: string (nullable = true)
 |-- Origin_State: string (nullable = true)
 |-- Origin_Lat: double (nullable = true)
 |-- Origin_Long: double (nullable = true)
 |-- Dest_City: string (nullable = true)
 |-- Dest_State: string (nullable = true)
 |-- Dest_Lat: double (nul

+----+--------+-------+---------+
|Year|PlaneAge|DepHour|IsWeekend|
+----+--------+-------+---------+
|2006|       7|     10|        0|
|2006|       7|     18|        0|
|2006|       6|     14|        0|
|2006|       7|     18|        0|
|2006|       7|     14|        0|
+----+--------+-------+---------+
only showing top 5 rows


In [47]:
# ==========================================================
# Column Reordering
# ==========================================================

# Grouping columns by logic: Temporal, Operational, Geographic, and Metrics
ordered_columns = [
    # 1. Temporal Features
    "Year", "Month", "DayofMonth", "DayOfWeek", "IsWeekend", "DepHour",
    
    # 2. Flight & Aircraft Information
    "UniqueCarrier", "Distance", "PlaneManufacturer", "PlaneAge",
    
    # 3. Schedule Information
    "CRSDepTime", "CRSArrTime", "CRSElapsedTime",
    
    # 4. Origin Geography
    "Origin", "Origin_City", "Origin_State", "Origin_Lat", "Origin_Long",
    
    # 5. Destination Geography
    "Dest", "Dest_City", "Dest_State", "Dest_Lat", "Dest_Long",
    
    # 6. Real-time Metrics and Target Variable
    "DepTime", "DepDelay", "TaxiOut", "ArrDelay"
]

# Apply the selection
enriched_df = enriched_df.select(*ordered_columns)

print("--- Dataframe successfully reordered ---")
enriched_df.limit(5).show()

--- Dataframe successfully reordered ---


+----+-----+----------+---------+---------+-------+-------------+--------+-----------------+--------+----------+----------+--------------+------+-----------+------------+-----------+------------+----+---------+----------+-----------+------------+-------+--------+-------+--------+
|Year|Month|DayofMonth|DayOfWeek|IsWeekend|DepHour|UniqueCarrier|Distance|PlaneManufacturer|PlaneAge|CRSDepTime|CRSArrTime|CRSElapsedTime|Origin|Origin_City|Origin_State| Origin_Lat| Origin_Long|Dest|Dest_City|Dest_State|   Dest_Lat|   Dest_Long|DepTime|DepDelay|TaxiOut|ArrDelay|
+----+-----+----------+---------+---------+-------+-------------+--------+-----------------+--------+----------+----------+--------------+------+-----------+------------+-----------+------------+----+---------+----------+-----------+------------+-------+--------+-------+--------+
|2006|    1|         2|        1|        0|     10|           XE|     339|          EMBRAER|       7|      1040|      1200|          80.0|   ABE|  Allentown|

# CHECKPOINT: DELETE BEFORE SUBMIT

In [48]:
enriched_df.write.mode("overwrite").parquet("../check_point_v1")

# Punto de Carga


In [ ]:
# 2. Re-cargas (esto crea un nuevo punto de partida limpio)
enriched_df = spark.read.parquet("../check_point_v1")

### EDA and statistical analysis over these features
*(Proper exploratory data analysis (possibly including univariate and/or multivariate
analysis) to better understand the input data and provide robust criteria for variable
selection.)*

### Feature Selection


### PCA ???

## Code used to train, test and save the model.

### Scaling of values ? 

### Definition of models to employ and metrics to use 
* *Select more than one valid machine learning algorithm for building the model.*
* *Consider more than one possible model performance metric and explain the criteria for
selecting the most appropriate*

### Cross-Validation loop to validate models with 80/20
* *Use cross-validation techniques to select the best model*
* *Perform model hyper-parameter tuning*
* *Use the full capacities of Spark’s MLlib*


### Selection and saving the best model 